In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
glove_wiki_vectors.most_similar("green")

[('red', 0.8204777836799622),
 ('blue', 0.7868045568466187),
 ('brown', 0.7808514833450317),
 ('yellow', 0.7705442905426025),
 ('white', 0.7624269723892212),
 ('gray', 0.7491011023521423),
 ('black', 0.7330776453018188),
 ('pink', 0.7289762496948242),
 ('purple', 0.7276665568351746),
 ('orange', 0.7111418843269348)]

In [7]:
glove_wiki_vectors.most_similar("lemon")

[('lime', 0.8365369439125061),
 ('juice', 0.8233426213264465),
 ('ginger', 0.766997754573822),
 ('zest', 0.7490259408950806),
 ('pepper', 0.7483124732971191),
 ('vinegar', 0.7325056791305542),
 ('cream', 0.726084291934967),
 ('garlic', 0.7254719734191895),
 ('olive', 0.7217621207237244),
 ('vanilla', 0.716948926448822)]

In [8]:
glove_wiki_vectors.most_similar("turtle")

[('turtles', 0.7838327884674072),
 ('leatherback', 0.687718391418457),
 ('tortoise', 0.6861258745193481),
 ('crab', 0.6687701344490051),
 ('alligator', 0.6632605195045471),
 ('shark', 0.6445285081863403),
 ('dolphin', 0.6353225708007812),
 ('loggerhead', 0.634040355682373),
 ('hawksbill', 0.6306670904159546),
 ('quail', 0.6191232800483704)]

In [9]:
glove_wiki_vectors.most_similar("donut")

[('doughnut', 0.7872991561889648),
 ("dunkin'", 0.6363549828529358),
 ('donuts', 0.6240694522857666),
 ('bagel', 0.5640051364898682),
 ('shack', 0.5480520129203796),
 ('krispy', 0.5395475029945374),
 ('kreme', 0.5271188020706177),
 ('doughnuts', 0.5165933966636658),
 ('peppermint', 0.5074476599693298),
 ('trinket', 0.49677151441574097)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [10]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [11]:
coast_shore_similarity = glove_wiki_vectors.similarity("coast", "shore")
clothes_closet_similarity = glove_wiki_vectors.similarity("clothes", "closet")
old_new_similarity = glove_wiki_vectors.similarity("old", "new")
smart_intelligent_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
dog_cat_similarity = glove_wiki_vectors.similarity("dog", "cat")
tree_lawyer_similarity = glove_wiki_vectors.similarity("tree", "lawyer")

In [12]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [13]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [14]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [15]:
glove_wiki_vectors.similarity("white", "rich")

0.44723597

In [16]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

In [17]:
wordA = "poor" # bias word
wordB = "white" # group 1
wordC = "black" # group 2

biasAB = glove_wiki_vectors.similarity(wordA, wordB)
biasAC = glove_wiki_vectors.similarity(wordA, wordC)

print(f"The association between {wordA} and {wordB} is: {biasAB} \nThe association between {wordA} and {wordC} is: {biasAC}")

The association between poor and white is: 0.44083836674690247 
The association between poor and black is: 0.5077115297317505


In [18]:
wordD = "nice" # bias word
wordE = "man" # group 1
wordF = "woman" # group 2

biasDE = glove_wiki_vectors.similarity(wordD, wordE)
biasDF = glove_wiki_vectors.similarity(wordD, wordF)

print(f"The association between {wordD} and {wordE} is: {biasDE} \nThe association between {wordD} and {wordF} is: {biasDF}")

The association between nice and man is: 0.47783035039901733 
The association between nice and woman is: 0.3721133768558502


In [19]:
wordG = "ugly" # bias word
wordH = "boy" # group 1
wordI = "girl" # group 2

biasGH = glove_wiki_vectors.similarity(wordG, wordH)
biasGI = glove_wiki_vectors.similarity(wordG, wordI)

print(f"The association between {wordG} and {wordH} is: {biasGH} \nThe association between {wordG} and {wordI} is: {biasGI}")

The association between ugly and boy is: 0.43425217270851135 
The association between ugly and girl is: 0.47462984919548035


In [20]:
wordJ = "useful" # bias word
wordK = "senior" # group 1
wordL = "adult" # group 2

biasJK = glove_wiki_vectors.similarity(wordJ, wordK)
biasJL = glove_wiki_vectors.similarity(wordJ, wordL)

print(f"The association between {wordJ} and {wordK} is: {biasJK} \nThe association between {wordJ} and {wordL} is: {biasJL}")

The association between useful and senior is: 0.2444399893283844 
The association between useful and adult is: 0.34693124890327454


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

1. There does appear to be some bias in the embedding. 

"poor" is roughly equally associated with "white" and "black" are comparable, so I wouldn't be too worried about bias there. 

"nice" is quite a bit more associated with "man" as opposed to "woman". This is actually the opposite of the bias I expected, but does point out there may be gender related biases still present in the embedding. 

"ugly" is fairly equally associated with "boy" and "girl", so no bias there. 

"useful" is quite a bit more associated with "adult" as opposed to "senior", indicating we may want to be aware of any ageism in the embedding. 

Overall this embedding is doing fairly well at minimizing bias, although there does appear to still be some present. While I wouldn't outright refuse to use the model, I would certainly be careful about it's use cases, and interpreting results from using it. 

2. These biases can be harmful in a number of ways. Imagine if a young girl was using this model in some way to try to plan her career, and the model was reinforcing gender stereotypes by pushing her towards more traditionally 'feminine' roles instead of what she actually wants to pursue. Likewise, it could be outright discriminatory if this type of model was used by a company to assess candidates for an open position, if it sees certain genders as being more vavourable for certain roles. 

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [21]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [22]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [23]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [24]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

In [25]:
from sklearn.metrics import classification_report

pipe = make_pipeline(CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000))
pipe.fit(X_train, y_train)
bow_train_accuracy = pipe.score(X_train, y_train)
bow_test_accuracy = pipe.score(X_test, y_test)
bow_report = classification_report(y_test, pipe.predict(X_test), digits=2)

print(f"Train: {bow_train_accuracy} \nTest: {bow_test_accuracy}")
print(bow_report)

Train: 0.9562051178314959 
Test: 0.8173666823973572
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [26]:
# adapted from lecture 17
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [27]:
lr = LogisticRegression(max_iter=2000)
lr.fit(X_train_embeddings, y_train)

/Users/kaigroden-gilchrist/opt/anaconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=2000)

In [28]:
emb_train_accuracy = lr.score(X_train_embeddings, y_train)
emb_test_accuracy = lr.score(X_test_embeddings, y_test)
emb_report = classification_report(y_test, lr.predict(X_test_embeddings), digits=2)

print(f"Train: {emb_train_accuracy} \nTest: {emb_test_accuracy}")
print(emb_report)

Train: 0.8493981996561141 
Test: 0.7928268050967437
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.88      1423
         bonding       0.83      0.76      0.79       492
enjoy_the_moment       0.56      0.51      0.53       469
        exercise       0.67      0.73      0.70        74
         leisure       0.71      0.65      0.68       407
          nature       0.67      0.72      0.69        71

        accuracy                           0.79      4238
       macro avg       0.73      0.73      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



In [29]:
grader.check("q1.5")

q1.5 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

1. CountVectorizer encodes the occurance of each word (either as a count or binary present/absent), which is quick, easy, and interpretable, but does not take into account any syntax or structure of the text. Average-embedding encodes each word as a vector of relationships between words, giving a syntactic representation that the CountVectorizer does not, at the cost of losing interpretability and expensive training. 
2. The CountVectorizer is more interpretable. It's easy to understand whether a word is present/absent in a text, and the effect that has (given by lr coef) on a model's prediction. It's much less intuitive too look at the output of the embedding, which is a vast matrix of relationships, and especially difficult to understand how the model goes from that matrix to a prediction. 
3. Yes. The SpaCy en_core_web_md is a pre-trained model that we are importing to use on our own data. This is an example of transfer learning. In this example, it appears that using the pre-trained model is resulting in a worse test accuracy versus trainig our own (79% vs 81%, respectively). In general, the benefit to transfer learning is that language models are very expensive to train, and you'd need vast amounts of data, computational power, and time. Importing a pre-trained model saves this, although it does mean the model is quite general, and may not be accurate in specific use-cases. However, as mentioned, in this case the model we're training ourself is doing slightly better. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [30]:
from sklearn.datasets import fetch_20newsgroups

In [31]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [32]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

> First we'll define a preprocessing function:

In [33]:
# adapted from lecture 17
def preprocess(
    doc,
    min_token_len=2,
):
    clean_text = []

    for token in doc:
        if (
            token.is_alpha  # getting rid of slashes or other weird characters
            and token.like_url == False # getting rid of urls
            and token.like_email == False # getting rid of email addresses
            and token.is_stop == False # excluding stopwords
            and token.is_upper == False #exclude hockey team names which tend to take their own category separate from hockey
            and len(token) > min_token_len #exclude any insignificant small words
        ): 
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

> Here we can view the updated DataFrame:

In [34]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]
df.head()

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read sence wonder help provide paragraph ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late foreign ministry spokesman ferhat ataman ...


> And we can view a full example of the preprocessed feature:

In [35]:
df["text_pp"][1]

'bad question ref list algorithm think bit hard point find plane define circle lie algorithm find center circle line pass center perpendicular plane point pass center sphere repeat unused point original point give different line pass sphere origin interection center sphere radius easy compute distance center original point leave math workable algorithm alternate method pair point plane form perpendicular bisector line segment pair contain center sphere pair form plane intersect point easy implement'

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

The steps carried out were:

1. Eliminating all non alphabetic characters. This will remove any slashes, or other weird characters that can confuse the model. 
2. Remove URLs and emails. These are likely unique, and will confuse the model more than help.
3. Remove stopwords. These would be present in every piece of text and would increase computation time without benefitting the model. 
4. Removed upper case words. Specifically relevant to hockey team names, which tend to take their own category separate from hockey.
4. Filtered out any words shorter than 2 characters. These are usually general and not useful in these types of models. 
5. Lemmatize words for consistency between text samples. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [36]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

In [37]:
# adapted from lecture 17
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=50, random_state=1
)
document_topics = lda.fit_transform(X)

I chose 4 topics, since the original list of topics can be broadly grouped into 4 categories. 

Hockey and baseball are combined into "Sports"

Christian and atheism are combined into "Religion"

Computer graphics and Windows are combined into "Computers"

Middle east and guns are combined into "Politics"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [38]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [39]:
# adapted from lecture 17
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
god           people        file          game          
people        say           program       team          
think         gun           use           year          
know          know          image         play          
believe       armenian      window        player        
jesus         right         available     good          
say           kill          include       season        
thing         come          run           win           
time          time          server        think         
like          state         work          hockey        




The topics can clearly be identified as follows:

Topic 0: Religion

Topic 1: Politics

Topic 2: Computers

Topic 3: Sports

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [40]:
topics = ["topic 0", "topic 1", "topic 2", "topic 3"]
for i in range(5):
    idx = pd.Series(document_topics[i]).idxmax()
    assignment = topics[idx]
    confidence = document_topics[i].max()
    actual_topic = df['target_name'][i]
    print(f"Document {i+1}: Assignment = {assignment} || Confidence = {confidence} || Actual topic = {actual_topic}")

Document 1: Assignment = topic 1 || Confidence = 0.9844961138063366 || Actual topic = talk.politics.guns
Document 2: Assignment = topic 2 || Confidence = 0.9887907713674543 || Actual topic = comp.graphics
Document 3: Assignment = topic 0 || Confidence = 0.7281962253760645 || Actual topic = comp.graphics
Document 4: Assignment = topic 2 || Confidence = 0.8756394964730387 || Actual topic = comp.windows.x
Document 5: Assignment = topic 1 || Confidence = 0.9823750701012239 || Actual topic = talk.politics.mideast


Document 1 was assigned topic 1 (politics), and the actual topic was politics (guns).

Document 2 was assigned topic 2 (computers), and the actual topic was computers (graphics).

Document 3 was assigned topic 0 (religion), and the actual topic was computers (grapgics). This is incorrect, although the model was slightly less confident in this prediction. 

Document 4 was assigned topic 2 (computers), and the actual topic was computers (windows). 

Document 5 was assigned topic 1 (politics), and the actual topic was politics (middle-east). 

Overall the model was correct 4/5 times, which is quite good, and the only one it missed it was less confident in. Admittedly, 73% is still quite confident, but it is notably lower than the other predictions. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. 

 - Decision trees: is classified by scikit-learn as inherently multi-class, in that it supports One-Vs-One or One-Vs-The-Rest classification. The tree can branch to distinguish a single class either way.
 - KNN: is also classified by scikit-learn as inherently multi-class. The nearest neighbor can accomodate any number of classess, although with more than two you might end up with more ties. 
 - Random Forest: since it is an ensemble of trees, it is also inherently multi-class in the same way as decision trees. 
 - Logistic Regression: uses One-Vs-The-Rest approach by setting multi_class=”ovr. 
 - SVM RBF: Uses One-Vs-One approach, since the resulting matrices can only compare two elements. 
  

2. Transfer learning involves applying pre-trained models to make predictions on your data instead of training a new one from scratch. This has several advantages, including a reduced need for training data, and computational power to train. This is particularly important in NLP and computer vision models, where training is especially expensive. 

3. A neural network's sequential transformations allows it to adapt to data which may not have been specifically designed for it. This is particularly helpful for transfer learning, since early layers can be general and adaptable, and later layers can be added to move towards a specific prediction. 

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)